In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
# getting the raw table from the wikipage
results = requests.get(url).text
df = pd.read_html(results, header=0, attrs={"class":"wikitable sortable"})[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [4]:
# removing rows where Borough is 'Not Assigned'
df['Check'] = df.Borough == 'Not assigned'
df = df[df['Check'] == False]
df.drop(['Check'], axis=1,inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [5]:
# merging rows part of the same Postcode and Borough
df = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()

In [6]:
# checking row where Neighbourhood is 'Not Assigned'
df.loc[df.Postcode == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [7]:
# defining a function which will replace Not assigned with the Borough else will keep the neighbourhood
def neighb(x):
    if x['Neighbourhood'] == 'Not assigned':
        return x['Borough']
    else: 
        return x['Neighbourhood']

In [8]:
df.Neighbourhood = df.apply(neighb, axis=1)

In [9]:
df.loc[df.Postcode == 'M7A']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [10]:
df.shape

(103, 3)

In [11]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [12]:
df.to_csv('toronto.csv')